In [1]:
!pip install numpy sentence-transformers bertopic hdbscan nltk scann
import nltk
nltk.download('punkt')
import nltk
nltk.download('punkt_tab')
!pip install sentence-transformers bertopic hdbscan umap-learn scann nltk datasets
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scann 1.4.0 requires numpy~=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=

# Topic Generation

In [ ]:
# === IMPORTS & SETUP ===
import os
import random
import numpy as np
import torch
import nltk
import logging
import re

from collections import defaultdict, Counter
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP
import scann

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.metrics import silhouette_score, precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity

# === SEED FIXING ===
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True)
nltk.download("punkt")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# === CLEANING & CONTEXT EXTRACTION (IMPROVED) ===
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()


def extract_entity_contexts(chunks, entities_per_chunk, use_multi_sentence=True):
    entity_context_pairs = []
    for idx, ents in enumerate(entities_per_chunk):
        chunk = clean_text(chunks[idx])
        sentences = sent_tokenize(chunk)
        for ent in ents:
            ent_lower = ent.lower()
            matched = False
            for i, sent in enumerate(sentences):
                if ent_lower in sent:
                    context = " ".join(sentences[max(0, i - 1): i + 2]) if use_multi_sentence else sent.strip()
                    enriched = f"The concept '{ent_lower}' appears in the following context: {context}"
                    entity_context_pairs.append((ent_lower, enriched.strip()))
                    matched = True
                    break
            if not matched:
                fallback = f"The concept '{ent_lower}' appears in the following context: {chunk}"
                entity_context_pairs.append((ent_lower, fallback.strip()))
    return entity_context_pairs


# === TOPIC SEARCHER CLASS (WITH DEDUPLICATION, NOISE FILTERING) ===
class AllergyTopicSearcher:
    def __init__(self, chunks, entities_per_chunk, umap_params, hdbscan_params,
                 model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"):
        self.chunks = chunks
        self.entities_per_chunk = entities_per_chunk
        self.embedding_model = SentenceTransformer(model_name)

        self.umap_params = umap_params
        self.hdbscan_params = hdbscan_params

        self.topic_model = None
        self.topic_metadata = []
        self.topic_embeddings = None
        self.searcher = None

        self._prepare()

    def _prepare(self):
        entity_context_pairs = extract_entity_contexts(self.chunks, self.entities_per_chunk)
        contextual_texts = [ctx for _, ctx in entity_context_pairs]
        contextual_embeddings = self.embedding_model.encode(contextual_texts, normalize_embeddings=False)

        umap_model = UMAP(**self.umap_params, random_state=SEED)
        hdbscan_model = HDBSCAN(**self.hdbscan_params, prediction_data=True)

        self.topic_model = BERTopic(
            embedding_model=self.embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            representation_model=KeyBERTInspired(),
            calculate_probabilities=True,
            verbose=False,
        )

        topics, _ = self.topic_model.fit_transform(contextual_texts, embeddings=contextual_embeddings)

        topic_to_contexts = defaultdict(list)
        topic_to_entities = defaultdict(set)
        topic_to_embeddings = defaultdict(list)

        for i, topic in enumerate(topics):
            if topic == -1:
                continue  # Skip noisy topics
            ent, ctx = entity_context_pairs[i]
            topic_to_contexts[topic].append(ctx)
            topic_to_entities[topic].add(ent)
            topic_to_embeddings[topic].append(contextual_embeddings[i])

        topic_embeddings = []
        topic_metadata = []

        for topic_id in topic_to_contexts:
            emb = topic_to_embeddings[topic_id]
            centroid = np.mean(emb, axis=0)
            centroid /= np.linalg.norm(centroid) + 1e-10
            topic_embeddings.append(centroid)
            topic_metadata.append({
                "topic_id": topic_id,
                "entities": list(topic_to_entities[topic_id]),
                "sentences": topic_to_contexts[topic_id],
                "sentence_embeddings": np.array(emb)
            })

        # === OPTIONAL: Merge semantically similar topics (cosine sim > 0.95)
        deduped_metadata = []
        used = set()

        for i, emb_i in enumerate(topic_embeddings):
            if i in used:
                continue
            group = [i]
            sim_scores = cosine_similarity([emb_i], topic_embeddings)[0]
            for j in range(i + 1, len(sim_scores)):
                if sim_scores[j] > 0.95:
                    group.append(j)
                    used.add(j)

            merged = {
                "topic_id": i,
                "sentences": [],
                "entities": [],
                "sentence_embeddings": []
            }
            for g in group:
                merged["sentences"] += topic_metadata[g]["sentences"]
                merged["entities"] += topic_metadata[g]["entities"]
                merged["sentence_embeddings"] += list(topic_metadata[g]["sentence_embeddings"])

            merged["sentence_embeddings"] = np.array(merged["sentence_embeddings"])
            merged["entities"] = list(set(merged["entities"]))
            deduped_metadata.append(merged)

        self.topic_metadata = deduped_metadata
        self.topic_embeddings = np.array([
            np.mean(m["sentence_embeddings"], axis=0) /
            (np.linalg.norm(np.mean(m["sentence_embeddings"], axis=0)) + 1e-10)
            for m in deduped_metadata
        ])

        num_points = len(self.topic_embeddings)
        num_leaves = min(10, num_points)
        leaves_to_search = min(5, num_leaves)

        if num_points < 3:
            raise ValueError("Not enough topics to build a search index.")

        self.searcher = (
            scann.scann_ops_pybind.builder(self.topic_embeddings, 3, "dot_product")
            .tree(num_leaves=num_leaves, num_leaves_to_search=leaves_to_search, training_sample_size=num_points)
            .score_brute_force()
            .reorder(min(5, num_points))
            .build()
        )


    import re

    def search(self, query, top_k_topics=3, top_k_sents=3):
        query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
        neighbors, scores = self.searcher.search(query_emb, final_num_neighbors=top_k_topics)

        results = []
        prefix_pattern = r"^the concept '.*?' appears in (the following )?context:\s*"

        for i, idx in enumerate(neighbors):
            meta = self.topic_metadata[idx]
            topic_score = float(scores[i])

            # Deduplicate sentences
            seen = set()
            cleaned_sentences = []
            cleaned_embeddings = []

            for sent, emb in zip(meta["sentences"], meta["sentence_embeddings"]):
            # Apply regex to remove beginning prefix
                cleaned = re.sub(prefix_pattern, "", sent, flags=re.IGNORECASE).strip()

        # No duplicates
                if cleaned not in seen:
                    seen.add(cleaned)
                    cleaned_sentences.append(cleaned)
                    cleaned_embeddings.append(emb)

            if not cleaned_sentences:
                continue

            emb_array = np.array(cleaned_embeddings)
            sims = np.dot(emb_array / np.linalg.norm(emb_array, axis=1, keepdims=True), query_emb)
            top_ids = sims.argsort()[::-1][:top_k_sents]

            top_sents = [(cleaned_sentences[j], float(sims[j])) for j in top_ids]
            results.append({
            "topic_id": meta["topic_id"],
            "topic_score": topic_score,
            "entities": meta["entities"],
            "sentences": top_sents,
            })

        return results




# === METRICS ===
def compute_bertopic_coherence(topic_model, topic_metadata, topk=15):
    topics = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    word_lists = [[word for word, _ in topic] for topic in topics]

    texts = []
    for meta in topic_metadata:
        for s in meta["sentences"]:
            tokens = clean_text(s).split()
            texts.append(tokens)

    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(t) for t in texts]

    cm = CoherenceModel(
        topics=word_lists,
        texts=texts,
        dictionary=dictionary,
        coherence="c_v"
    )
    return cm.get_coherence()


def compute_topic_diversity(topic_model, topic_metadata, topk=10):
    topic_words = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    unique_words = set(word for topic in topic_words for word, _ in topic)
    return len(unique_words) / (len(topic_words) * topk)


def compute_silhouette_score_custom(topic_metadata):
    all_embeddings = []
    all_labels = []

    for meta in topic_metadata:
        emb = meta["sentence_embeddings"]
        if len(emb) < 2:  # skip small clusters
            continue
        all_embeddings.extend(emb)
        all_labels.extend([meta["topic_id"]] * len(emb))

    if len(all_embeddings) < 3:
        return None

    all_embeddings = np.vstack(all_embeddings)
    return silhouette_score(all_embeddings, all_labels, metric="cosine")


# === DATASET & INITIALIZATION ===
allergy_dataset = {
  "chunks": [
  "Mr. James H., a 79-year-old male with a long-standing history of cardiovascular and metabolic diseases, was brought to the emergency department due to acute confusion and generalized weakness.",
  "According to his daughter, he had wandered outside disoriented and was unable to identify family members or recall events from the previous day.",
  "He has known medical conditions including hypertension, heart failure with reduced ejection fraction, insulin-dependent diabetes mellitus, stage 4 chronic kidney disease, and major depressive disorder.",
  "His medication regimen includes daily doses of lisinopril, furosemide, carvedilol, insulin glargine, sertraline, and donepezil.",
  "In the past 24 hours, the patient experienced decreased appetite, an episode of vomiting, and two instances of urinary incontinence.",
  "Vital signs upon arrival included a blood pressure of 98/56 mmHg, heart rate of 112 beats per minute (irregularly irregular), respiratory rate of 20, oxygen saturation of 93% on room air, and a temperature of 37.6°C.",
  "Physical examination revealed dry mucous membranes, poor skin turgor, moderate lower limb pitting edema, and delayed capillary refill.",
  "Auscultation of the lungs revealed bilateral basal crackles, and cardiac exam confirmed an irregularly irregular heartbeat without murmurs.",
  "Neurological examination showed fluctuating attention span but no signs of focal deficits or lateralizing neurological signs.",
  "Initial lab studies demonstrated an elevated blood glucose of 421 mg/dL, serum sodium of 129 mmol/L, potassium at 5.7 mmol/L, and creatinine at 2.9 mg/dL.",
  "Serum BUN was elevated at 59 mg/dL and the patient’s anion gap was calculated to be 19, consistent with an anion-gap metabolic acidosis.",
  "Urinalysis revealed glucosuria and ketonuria without signs of infection, and serum ketones were modestly elevated.",
  "His HbA1c on record from two months ago was 8.1%, confirming chronic poor glycemic control.",
  "An ECG showed atrial fibrillation with rapid ventricular response but no acute ischemic changes.",
  "Chest radiograph revealed cardiomegaly and pulmonary vascular congestion with mild bilateral pleural effusions.",
  "CT head without contrast was negative for acute infarct, hemorrhage, or mass effect, but showed chronic microvascular changes.",
  "Given the presentation, he was admitted to the medical ward for acute hyperosmolar hyperglycemic state (HHS) and acute on chronic kidney injury.",
  "A diagnosis of acute delirium, likely secondary to metabolic derangements, volume depletion, and possible infection, was made.",
  "He was started on intravenous normal saline, correctional insulin, and telemetry monitoring.",
  "Furosemide was temporarily held due to volume depletion, and electrolytes were repleted cautiously under nephrology guidance.",
  "Blood cultures, urine cultures, and chest x-ray were obtained to rule out infection as a potential delirium trigger.",
  "Empiric antibiotics (ceftriaxone and azithromycin) were initiated pending culture data due to concern for possible aspiration pneumonia.",
  "On day two, the patient’s mental status began to improve with the resolution of hyperglycemia and normalization of serum osmolarity.",
  "Repeat labs showed trending down of BUN and creatinine, with sodium rising to 134 and potassium corrected to 4.5 mmol/L.",
  "He remained in atrial fibrillation and required continuation of beta-blocker therapy to manage ventricular rate.",
  "Apixaban was continued upon nephrology clearance given acceptable bleeding risk and stable renal function.",
  "He was evaluated by geriatrics for worsening cognitive decline and safety evaluation related to home discharge.",
  "PT/OT performed a bedside mobility assessment showing weakness, unsteadiness, and need for moderate assistance with transfers.",
  "Case management consulted social work regarding home safety, fall prevention, and caregiver support.",
  "His hospital stay was complicated by mild hypoglycemia on hospital day 3, prompting insulin dose adjustments.",
  "Nutritional support was consulted to optimize diabetic-friendly, renal-adjusted diet appropriate for age and mobility.",
  "His depression management was reviewed with psychiatry, and sertraline was continued at 100 mg/day with no suggestion for dose change.",
  "A Montreal Cognitive Assessment (MoCA) was done revealing a score of 19/30, indicating significant mild cognitive impairment.",
  "Audiology was recommended due to hearing difficulty interfering with care discussions.",
  "Oral exam noted poor dentition; dental evaluation was recommended for follow-up to address suspected pain and poor appetite.",
  "After 6 days, the patient was clinically improved, mentally oriented, and ambulatory with the help of physical therapy.",
  "Cardiac and renal parameters stabilized sufficiently to permit safe discharge planning.",
  "The final hospital diagnosis included hyperosmolar hyperglycemic state, volume depletion, acute-on-chronic kidney injury, atrial fibrillation with RVR, and acute delirium.",
  "He was discharged on a simplified diabetic regimen including basal insulin and correctional sliding scale doses only.",
  "Apixaban, carvedilol, donepezil, and sertraline were continued with no changes.",
  "Discharge medication reconciliation included temporary hold of furosemide with plan for outpatient reassessment after fluid status recovery.",
  "Caregiver role was assumed by daughter who had durable power of attorney and assisted with all home-based needs.",
  "Written instructions and red flags for hyperglycemia, dizziness, and recurrent confusion were provided.",
  "A follow-up with his primary care physician, nephrologist, and endocrinologist were scheduled within one and two weeks respectively.",
  "Home health nursing was arranged to provide medication support and glucose monitoring.",
  "Nutritionist and physical therapy were ordered for continued improvement in diet and mobility.",
  "Advanced care planning was briefly discussed including code status, proxy, and end-of-life preferences.",
  "He is currently listed as full code but family is open to further discussion at next provider visit.",
  "Patient was grateful for hospital care and expressed motivation to remain active and well at home.",
  "The overall prognosis remains guarded due to progressive cognitive decline and limited renal reserve.",
  "Close monitoring for new signs of decompensation or medication nonadherence was advised.",
  "Pulmonology follow-up was discussed due to prior mild restrictive spirometry suggestive of early interstitial lung disease.",
  "Family history reveals mother died of complications from dementia and father from ischemic stroke.",
  "No reported use of tobacco, alcohol, or recreational drugs throughout his life.",
  "Lives in a single-story home with grab bars and minimal clutter, although risks for falls still persist.",
  "Wears eyeglasses but rarely uses his hearing aids, sometimes leading to miscommunication or withdrawal.",
  "History of previous admission 1 year ago for pneumonia requiring IV antibiotics and 6-day hospitalization.",
  "Documentation from that admission revealed transient delirium and impaired oral intake similar to current episode.",
  "Goals-of-care conversations were initiated during this admission but deferred for primary care setting follow-up.",
  "Social isolation remains a concern, especially since his wife passed away 3 years ago.",
  "Patient receives Meals on Wheels but misses many meal deliveries due to lack of reliable caregiver at times.",
  "Transportation to medical appointments is provided by his daughter, who balances full-time work responsibilities.",
  "No current enrollment in adult day health programs; options discussed with case management on discharge.",
  "Insurance covers home nursing and outpatient labs but does not cover custodial care.",
  "Patient was educated about Medicare Advantage benefits and encouraged to review covered services with the plan coordinator.",
  "He was also reminded of the importance of daily glucose checks and hydration in summer months.",
  "Foot exam demonstrated mild calluses and intact sensation; he denies new ulcers or foot injuries.",
  "Vaccination status confirmed: received influenza and COVID vaccines last fall, but is due for pneumococcal booster.",
  "Dentition issues may be contributing to decreased intake; dental clinic referral was sent through EHR.",
  "Assistive device for walking was provided (four-point cane) after physical therapy evaluation.",
  "Contact dermatitis on legs due to prolonged pressure and incontinence was treated with barrier cream.",
  "Skin care and bathing guidance were reviewed with family nursing staff prior to discharge.",
  "Patient verbalized understanding of all discharge instructions with support from daughter.",
  "Hospital team closed chart after discussing active problems list, response to therapy, and continued plan.",
  "Patient left the hospital in a wheelchair, accompanied by family, and appeared in good spirits.",
  "The full discharge plan was documented and faxed to his primary provider for continuity of care.",
  "Medication reconciliation showed no potential drug interactions or allergy mismatches.",
  "He was warned against use of NSAIDs due to underlying CKD and risk of acute worsening.",
  "Hydration goals of at least 1.5 liters per day were set; urination logs and symptom review were encouraged.",
  "Emergency instructions included what to do in case of unresponsiveness, low blood glucose, sudden confusion, or chest pain.",
  "Digital blood glucose monitor was reviewed at bedside; daughter demonstrated appropriate calibration and use.",
  "All prescriptions were sent electronically to their local pharmacy located eight blocks from their home.",
  "Patient prefers morning appointments due to increased alertness and energy early in the day.",
  "A follow-up MoCA test was recommended in 3–6 months to assess cognitive trajectory.",
  "Updated advance directives were placed in the chart and a copy was given to the daughter.",
  "Fall prevention strategies were emphasized including appropriate lighting, footwear, and scheduled ambulation.",
  "Use of automatic pill organizers was encouraged to improve adherence across complex medication schedules.",
  "Daily weights will be tracked at home to monitor for unexpected fluid retention or heart failure.",
  "Serum creatinine will be rechecked in one week given borderline rise during admission.",
  "A nephrology note was sent to alert about potential need for long-term planning if GFR continues to decline.",
  "Patient qualifies for shared savings Medicare model and was assigned a care coordinator temporarily.",
  "Patient support group information was handed out, including resources for caregivers.",
  "He is open to exploring telehealth check-ins for medication titration and early symptom triage.",
  "Daughter confirmed she has portal access to review labs and visit summaries on his behalf.",
  "Patient and daughter expressed appreciation for the hospital care coordination team.",
  "Case closed with summary of diagnosis, medications, specialists involved, and plan for 30-day transitional care.",
  "Status post discharge: stable, safe for home, alert and oriented with supervision."
]

,
"entities":[
  ["confusion", "weakness", "cardiovascular", "metabolic"],
  ["disorientation", "memory"],
  ["hypertension", "failure", "diabetes", "kidney", "depression"],
  ["lisinopril", "furosemide", "carvedilol", "insulin", "sertraline", "donepezil"],
  ["appetite", "vomiting", "incontinence"],
  ["pressure", "rate", "rhythm", "respiration", "saturation", "temperature"],
  ["mucosa", "turgor", "edema", "refill"],
  ["crackles", "heartbeat", "murmurs"],
  ["attention", "deficits"],
  ["glucose", "sodium", "potassium", "creatinine"],
  ["bun", "acidosis"],
  ["glucosuria", "ketonuria", "ketones"],
  ["hba1c", "control"],
  ["ecg", "fibrillation", "response", "ischemia"],
  ["cardiomegaly", "congestion", "effusions"],
  ["infarct", "hemorrhage", "microvascular"],
  ["hyperglycemia", "injury"],
  ["delirium", "derangements", "infection"],
  ["saline", "insulin", "telemetry"],
  ["furosemide", "depletion", "electrolytes"],
  ["cultures", "infection"],
  ["antibiotics", "ceftriaxone", "azithromycin", "pneumonia"],
  ["status", "hyperglycemia", "osmolarity"],
  ["bun", "creatinine", "sodium", "potassium"],
  ["fibrillation", "rate", "blocker"],
  ["apixaban", "function", "bleeding"],
  ["geriatrics", "cognition"],
  ["pt", "ot", "mobility", "weakness", "transfers"],
  ["safety", "falls"],
  ["hypoglycemia", "insulin"],
  ["nutrition", "diet"],
  ["depression", "psychiatry", "sertraline"],
  ["moca", "impairment"],
  ["audiology", "hearing"],
  ["dentition", "pain"],
  ["therapy", "ambulation"],
  ["parameters"],
  ["hyperglycemia", "depletion", "injury", "fibrillation", "delirium"],
  ["regimen", "insulin"],
  ["apixaban", "carvedilol", "donepezil", "sertraline"],
  ["reconciliation", "furosemide"],
  ["power"],
  ["hyperglycemia", "dizziness", "confusion"],
  ["nephrologist", "endocrinologist"],
  ["nursing", "glucose"],
  ["nutritionist", "therapy"],
  ["planning", "status", "proxy"],
  ["code"],
  ["prognosis", "cognition", "reserve"],
  ["monitoring", "decompensation", "adherence"],
  ["pulmonology", "spirometry", "disease"],
  ["dementia", "stroke"],
  ["tobacco", "alcohol", "drugs"],
  ["falls"],
  ["hearing"],
  ["pneumonia", "antibiotics"],
  ["delirium"],
  ["conversations"],
  ["isolation"],
  ["meals"],
  ["transportation"],
  ["enrollment"],
  ["insurance", "nursing", "labs"],
  ["medicare"],
  ["glucose", "hydration"],
  ["exam", "calluses", "ulcers"],
  ["vaccination", "influenza", "covid", "booster"],
  ["dentition", "referral"],
  ["cane"],
  ["dermatitis", "cream"],
  ["skin"],
  ["instructions"],
  ["problems", "therapy", "plan"],
  ["wheelchair"],
  ["continuity"],
  ["reconciliation", "interactions", "allergies"],
  ["nsaids"],
  ["hydration", "urination", "symptoms"],
  ["instructions", "glucose", "confusion", "pain"],
  ["monitor", "calibration"],
  ["prescriptions", "pharmacy"],
  ["appointments", "alertness", "energy"],
  ["moca"],
  ["directives"],
  ["prevention", "lighting", "footwear", "ambulation"],
  ["organizer", "adherence"],
  ["weight", "retention", "failure"],
  ["creatinine"],
  ["nephrology", "gfr"],
  ["medicare", "coordinator"],
  ["group", "caregivers"],
  ["telehealth", "titration", "triage"],
  ["portal", "labs", "summaries"],
  ["coordination"],
  ["diagnosis", "medications", "specialists", "care"],
  ["discharge", "supervision"]
]



}

from itertools import product
import time

# === HYPERPARAMETER GRID SEARCH ===
def grid_search_best_params(chunks, entities_per_chunk, umap_param_grid, hdbscan_param_grid):
    best_score = -np.inf
    best_umap = None
    best_hdbscan = None
    best_metrics = None

    combinations = list(product(umap_param_grid["n_neighbors"],
                                umap_param_grid["n_components"],
                                umap_param_grid["min_dist"],
                                hdbscan_param_grid["min_cluster_size"],
                                hdbscan_param_grid["min_samples"]))

    logger.info(f"Starting grid search over {len(combinations)} combinations...")

    for (n_neighbors, n_components, min_dist, min_cluster_size, min_samples) in combinations:
        umap_params = {
            "n_neighbors": n_neighbors,
            "n_components": n_components,
            "min_dist": min_dist,
            "metric": "cosine"
        }
        hdbscan_params = {
            "min_cluster_size": min_cluster_size,
            "min_samples": min_samples,
            "metric": "euclidean"
        }

        try:
            start_time = time.time()
            model = AllergyTopicSearcher(chunks, entities_per_chunk, umap_params, hdbscan_params)
            coherence = compute_bertopic_coherence(model.topic_model, model.topic_metadata)
            diversity = compute_topic_diversity(model.topic_model, model.topic_metadata)
            silhouette = compute_silhouette_score_custom(model.topic_metadata) or 0.0

            # Composite scoring (tweak weights if desired)
            score = coherence + 0.5 * diversity + 0.5 * silhouette
            elapsed = time.time() - start_time

            logger.info(
                f" Tried UMAP: {umap_params}, HDBSCAN: {hdbscan_params} "
                f"=> Score: {score:.4f} (Coherence: {coherence:.4f}, Diversity: {diversity:.4f}, Silhouette: {silhouette:.4f}) - Time: {elapsed:.2f}s"
            )

            if score > best_score:
                best_score = score
                best_umap = umap_params
                best_hdbscan = hdbscan_params
                best_metrics = (coherence, diversity, silhouette)

        except Exception as e:
            logger.warning(f"Skipping combination due to error: {e}")

    return best_umap, best_hdbscan, best_metrics
# Define parameter grids
umap_grid = {
    "n_neighbors": [5, 10],
    "n_components": [3, 5],
    "min_dist": [0.1, 0.25]
}
hdbscan_grid = {
    "min_cluster_size": [2, 5],
    "min_samples": [1, 3]
}

# Run grid search
best_umap, best_hdbscan, best_metrics = grid_search_best_params(
    allergy_dataset["chunks"],
    allergy_dataset["entities"],
    umap_param_grid=umap_grid,
    hdbscan_param_grid=hdbscan_grid
)

print("\n=== Best Hyperparameters Found ===")
print(f"Best UMAP: {best_umap}")
print(f"Best HDBSCAN: {best_hdbscan}")
print(f"Metrics => Coherence: {best_metrics[0]:.4f}, Diversity: {best_metrics[1]:.4f}, Silhouette: {best_metrics[2]:.4f}")


searcher = AllergyTopicSearcher(
    chunks=allergy_dataset["chunks"],
    entities_per_chunk=allergy_dataset["entities"],
    umap_params=best_umap,
    hdbscan_params=best_hdbscan,
    model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)
print("Model ready for querying.")
# print("\n=== Generated Topics and Entities ===")
# for meta in searcher.topic_metadata:
#     topic_id = meta["topic_id"]
#     entities = ", ".join(meta["entities"])
#     print(f"Topic ID: {topic_id} — Entities: {entities}")

# === METRICS ===
coherence = compute_bertopic_coherence(searcher.topic_model, searcher.topic_metadata, topk=15)
diversity = compute_topic_diversity(searcher.topic_model, searcher.topic_metadata, topk=10)
sil_score = compute_silhouette_score_custom(searcher.topic_metadata)

print("\n=== Topic Quality Metrics ===")
print(f"Coherence Score (c_v): {coherence:.4f}")
print(f"Topic Diversity: {diversity:.4f}")
if sil_score is not None:
    print(f" Silhouette Score: {sil_score:.4f}")
else:
    print("Silhouette Score: Not applicable.")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== Best Hyperparameters Found ===
Best UMAP: {'n_neighbors': 10, 'n_components': 3, 'min_dist': 0.25, 'metric': 'cosine'}
Best HDBSCAN: {'min_cluster_size': 2, 'min_samples': 1, 'metric': 'euclidean'}
Metrics => Coherence: 0.6393, Diversity: 0.4465, Silhouette: 0.5239
Model ready for querying.

=== Topic Quality Metrics ===
Coherence Score (c_v): 0.6393
Topic Diversity: 0.4465
 Silhouette Score: 0.5239


In [ ]:
# === IMPORTS & SETUP ===
import os
import random
import numpy as np
import torch
import nltk
import logging
import re

from collections import defaultdict, Counter
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP
import scann

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.metrics import silhouette_score, precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity

# === SEED FIXING ===
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True)
nltk.download("punkt")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# === CLEANING & CONTEXT EXTRACTION (IMPROVED) ===
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()


def extract_entity_contexts(chunks, entities_per_chunk, use_multi_sentence=True):
    entity_context_pairs = []
    for idx, ents in enumerate(entities_per_chunk):
        chunk = clean_text(chunks[idx])
        sentences = sent_tokenize(chunk)
        for ent in ents:
            ent_lower = ent.lower()
            matched = False
            for i, sent in enumerate(sentences):
                if ent_lower in sent:
                    context = " ".join(sentences[max(0, i - 1): i + 2]) if use_multi_sentence else sent.strip()
                    enriched = f"The concept '{ent_lower}' appears in the following context: {context}"
                    entity_context_pairs.append((ent_lower, enriched.strip()))
                    matched = True
                    break
            if not matched:
                fallback = f"The concept '{ent_lower}' appears in the following context: {chunk}"
                entity_context_pairs.append((ent_lower, fallback.strip()))
    return entity_context_pairs


# === TOPIC SEARCHER CLASS (WITH DEDUPLICATION, NOISE FILTERING) ===
class AllergyTopicSearcher:
    def __init__(self, chunks, entities_per_chunk, umap_params, hdbscan_params,
                 model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"):
        self.chunks = chunks
        self.entities_per_chunk = entities_per_chunk
        self.embedding_model = SentenceTransformer(model_name)

        self.umap_params = umap_params
        self.hdbscan_params = hdbscan_params

        self.topic_model = None
        self.topic_metadata = []
        self.topic_embeddings = None
        self.searcher = None

        self._prepare()

    def _prepare(self):
        entity_context_pairs = extract_entity_contexts(self.chunks, self.entities_per_chunk)
        contextual_texts = [ctx for _, ctx in entity_context_pairs]
        contextual_embeddings = self.embedding_model.encode(contextual_texts, normalize_embeddings=False)

        umap_model = UMAP(**self.umap_params, random_state=SEED)
        hdbscan_model = HDBSCAN(**self.hdbscan_params, prediction_data=True)

        self.topic_model = BERTopic(
            embedding_model=self.embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            representation_model=KeyBERTInspired(),
            calculate_probabilities=True,
            verbose=False,
        )

        topics, _ = self.topic_model.fit_transform(contextual_texts, embeddings=contextual_embeddings)

        topic_to_contexts = defaultdict(list)
        topic_to_entities = defaultdict(set)
        topic_to_embeddings = defaultdict(list)

        for i, topic in enumerate(topics):
            if topic == -1:
                continue  # Skip noisy topics
            ent, ctx = entity_context_pairs[i]
            topic_to_contexts[topic].append(ctx)
            topic_to_entities[topic].add(ent)
            topic_to_embeddings[topic].append(contextual_embeddings[i])

        topic_embeddings = []
        topic_metadata = []

        for topic_id in topic_to_contexts:
            emb = topic_to_embeddings[topic_id]
            centroid = np.mean(emb, axis=0)
            centroid /= np.linalg.norm(centroid) + 1e-10
            topic_embeddings.append(centroid)
            topic_metadata.append({
                "topic_id": topic_id,
                "entities": list(topic_to_entities[topic_id]),
                "sentences": topic_to_contexts[topic_id],
                "sentence_embeddings": np.array(emb)
            })

        # === OPTIONAL: Merge semantically similar topics (cosine sim > 0.95)
        deduped_metadata = []
        used = set()

        for i, emb_i in enumerate(topic_embeddings):
            if i in used:
                continue
            group = [i]
            sim_scores = cosine_similarity([emb_i], topic_embeddings)[0]
            for j in range(i + 1, len(sim_scores)):
                if sim_scores[j] > 0.95:
                    group.append(j)
                    used.add(j)

            merged = {
                "topic_id": i,
                "sentences": [],
                "entities": [],
                "sentence_embeddings": []
            }
            for g in group:
                merged["sentences"] += topic_metadata[g]["sentences"]
                merged["entities"] += topic_metadata[g]["entities"]
                merged["sentence_embeddings"] += list(topic_metadata[g]["sentence_embeddings"])

            merged["sentence_embeddings"] = np.array(merged["sentence_embeddings"])
            merged["entities"] = list(set(merged["entities"]))
            deduped_metadata.append(merged)

        self.topic_metadata = deduped_metadata
        self.topic_embeddings = np.array([
            np.mean(m["sentence_embeddings"], axis=0) /
            (np.linalg.norm(np.mean(m["sentence_embeddings"], axis=0)) + 1e-10)
            for m in deduped_metadata
        ])

        num_points = len(self.topic_embeddings)
        num_leaves = min(10, num_points)
        leaves_to_search = min(5, num_leaves)

        if num_points < 3:
            raise ValueError("Not enough topics to build a search index.")

        self.searcher = (
            scann.scann_ops_pybind.builder(self.topic_embeddings, 3, "dot_product")
            .tree(num_leaves=num_leaves, num_leaves_to_search=leaves_to_search, training_sample_size=num_points)
            .score_brute_force()
            .reorder(min(5, num_points))
            .build()
        )


    import re

    def search(self, query, top_k_topics=3, top_k_sents=3):
        query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
        neighbors, scores = self.searcher.search(query_emb, final_num_neighbors=top_k_topics)

        results = []
        prefix_pattern = r"^the concept '.*?' appears in (the following )?context:\s*"

        for i, idx in enumerate(neighbors):
            meta = self.topic_metadata[idx]
            topic_score = float(scores[i])

            # Deduplicate sentences
            seen = set()
            cleaned_sentences = []
            cleaned_embeddings = []

            for sent, emb in zip(meta["sentences"], meta["sentence_embeddings"]):
            # Apply regex to remove beginning prefix
                cleaned = re.sub(prefix_pattern, "", sent, flags=re.IGNORECASE).strip()

        # No duplicates
                if cleaned not in seen:
                    seen.add(cleaned)
                    cleaned_sentences.append(cleaned)
                    cleaned_embeddings.append(emb)

            if not cleaned_sentences:
                continue

            emb_array = np.array(cleaned_embeddings)
            sims = np.dot(emb_array / np.linalg.norm(emb_array, axis=1, keepdims=True), query_emb)
            top_ids = sims.argsort()[::-1][:top_k_sents]

            top_sents = [(cleaned_sentences[j], float(sims[j])) for j in top_ids]
            results.append({
            "topic_id": meta["topic_id"],
            "topic_score": topic_score,
            "entities": meta["entities"],
            "sentences": top_sents,
            })

        return results




# === METRICS ===
def compute_bertopic_coherence(topic_model, topic_metadata, topk=15):
    topics = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    word_lists = [[word for word, _ in topic] for topic in topics]

    texts = []
    for meta in topic_metadata:
        for s in meta["sentences"]:
            tokens = clean_text(s).split()
            texts.append(tokens)

    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(t) for t in texts]

    cm = CoherenceModel(
        topics=word_lists,
        texts=texts,
        dictionary=dictionary,
        coherence="c_v"
    )
    return cm.get_coherence()


def compute_topic_diversity(topic_model, topic_metadata, topk=10):
    topic_words = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    unique_words = set(word for topic in topic_words for word, _ in topic)
    return len(unique_words) / (len(topic_words) * topk)


def compute_silhouette_score_custom(topic_metadata):
    all_embeddings = []
    all_labels = []

    for meta in topic_metadata:
        emb = meta["sentence_embeddings"]
        if len(emb) < 2:  # skip small clusters
            continue
        all_embeddings.extend(emb)
        all_labels.extend([meta["topic_id"]] * len(emb))

    if len(all_embeddings) < 3:
        return None

    all_embeddings = np.vstack(all_embeddings)
    return silhouette_score(all_embeddings, all_labels, metric="cosine")


# === DATASET & INITIALIZATION ===
allergy_dataset = {
  "chunks": ["A 58-year-old female with a history of invasive ductal carcinoma of the breast presented with unintentional weight loss and malaise over two months.",
  "She reported new onset of bone pain localized to the lower back and hips, progressively worsening in intensity.",
  "There was a history of intermittent fever and night sweats but no reported cough or hemoptysis.",
  "Her past medical history includes Stage II breast cancer treated with lumpectomy and adjuvant chemotherapy three years ago.",
  "She completed radiation therapy 18 months prior and was on hormonal therapy with tamoxifen.",
  "Family history revealed a mother diagnosed with ovarian cancer at age 62.",
  "Physical examination showed pallor, mild lymphadenopathy in the supraclavicular region, and tenderness over the lumbar spine.",
  "Vital signs were stable with a low-grade fever of 37.8°C documented.",
  "Neurological examination was non-focal with intact motor and sensory function.",
  "Skin inspection revealed scattered petechiae over the lower extremities.",
  "Initial laboratory tests showed anemia with a hemoglobin of 9.6 g/dL and elevated alkaline phosphatase.",
  "Serum calcium was elevated at 11.2 mg/dL with normal renal function.",
  "Liver enzymes revealed mildly elevated transaminases.",
  "Tumor markers demonstrated elevated CA 15-3 and carcinoembryonic antigen (CEA).",
  "Bone marrow biopsy was scheduled given pancytopenia and bone pain.",
  "Plain radiographs of the lumbar spine displayed lytic lesions involving vertebral bodies L3 to L5.",
  "A whole-body bone scan indicated multiple areas of increased uptake consistent with skeletal metastases.",
  "Contrast-enhanced CT scan of the chest, abdomen, and pelvis showed multiple hypodense liver lesions.",
  "No pulmonary nodules or mediastinal lymphadenopathy were identified.",
  "Ultrasound-guided liver biopsy confirmed metastatic adenocarcinoma consistent with a breast primary.",
  "Immunohistochemistry revealed estrogen and progesterone receptor positivity with HER2 negativity.",
  "The oncology team initiated treatment with a combination of bisphosphonates and aromatase inhibitors.",
  "Supportive care included analgesics for bone pain and intravenous hydration for hypercalcemia.",
  "She was referred to pain management and physiotherapy services early in the treatment course.",
  "The patient was counseled on prognosis, treatment goals, and potential side effects, emphasizing quality of life.",
  "Follow-up imaging after three cycles of systemic therapy showed partial response with reduction in liver lesions.",
  "Her hemoglobin levels improved gradually, and bone pain was better controlled after bisphosphonate therapy.",
  "Repeat tumor markers demonstrated a downward trend correlating with clinical improvement.",
  "She experienced mild nausea and fatigue as common adverse effects of aromatase inhibitors.",
  "Endocrinology was consulted to monitor bone mineral density and calcium homeostasis.",
  "Psychological support was provided as the patient reported anxiety and mood changes linked to diagnosis.",
  "Genetic counseling was offered due to family history and implications for targeted therapies and relatives.",
  "The patient was screened for vitamin D deficiency and started supplementation accordingly.",
  "Regular liver function tests and hematologic parameters were closely monitored throughout treatment.",
  "No evidence of brain metastases was found on screening MRI performed due to headaches.",
  "The multidisciplinary tumor board reviewed her case regularly to optimize management plans.",
  "Physical rehabilitation focused on maintaining mobility and preventing skeletal complications.",
  "She was advised on diet modifications to address nutritional deficits and maintain strength.",
  "Symptoms of potential pathological fractures were carefully monitored, with orthopedics in consultation.",
  "The patient's preferences strongly influenced decision-making, with goals centered on symptom control.",
  "Anemia was managed conservatively with erythropoiesis-stimulating agents as hemoglobin fell below threshold.",
  "The endocrinology team adjusted hormonal therapy due to emerging side effects including arthralgia.",
  "Bone biopsies excluded secondary malignancies and confirmed disease progression status.",
  "A plan for cyclic imaging surveillance was established to assess disease stability.",
  "The patient was enrolled in a clinical trial investigating novel agents for metastatic breast cancer.",
  "Treatment adherence was supported via nursing follow-ups and patient education materials.",
  "The role of palliative care was discussed early with emphasis on holistic support.",
  "Complications such as thrombocytopenia developed transiently, managed with dose adjustments.",
  "Cardiotoxicity surveillance was instituted due to prior chemotherapy exposure, with echocardiograms scheduled.",
  "The patient reported improved mood and coping strategies following psychiatric intervention.",
  "End-of-life care planning was briefly addressed with patient and family input incorporated.",
  "The patient participated actively in support groups facilitated by the oncology center.",
  "Pain control was optimized using a multimodal analgesic regimen including opioids and NSAIDs.",
  "Bone-modifying agents helped reduce skeletal related events and improve quality of life.",
  "Serum biomarkers were periodically assessed as part of response monitoring.",
  "The impact of hormonal therapy on bone health was managed with calcium and vitamin D supplements.",
  "Anemia-related fatigue was addressed with physical activity modulation and energy conservation techniques.",
  "The patient was closely observed for signs of disease progression or treatment resistance.",
  "Interdisciplinary communication between oncology, nursing, rehabilitation, and social workers was continuous.",
  "Access to financial and social support services was coordinated to assist with treatment costs.",
  "The patient’s diary was used to track symptoms and side effects to inform ongoing care.",
  "Reassessment of goals of care was planned regularly to align with changing clinical status.",
  "International guidelines for metastatic breast cancer management were applied in treatment decisions.",
  "The case exemplified challenges in balancing disease control with minimizing treatment toxicity.",
  "The patient was encouraged to engage in meaningful activities and social interactions.",
  "Long-term follow-up focused on symptom management, surveillance, and survivorship issues.",
  "The clinical summary contributed to educational efforts for trainees in complex cancer care.",
  "Transitions between inpatient, outpatient, and supportive community care were smoothly implemented.",
  "Family meetings facilitated shared decision-making and education about prognosis and care options.",
  "The clinical documentation included detailed medication reconciliation and allergy assessments.",
  "Data captured support future research and quality improvement initiatives in metastatic breast cancer.",
  "Final diagnoses included metastatic estrogen receptor positive breast cancer with skeletal and hepatic involvement.",
  "The patient’s overall prognosis was communicated with sensitivity and clarity aligning care expectations."
]

,
"entities":[
  ["female", "carcinoma", "breast", "weight", "malaise"],
  ["bone", "pain", "back", "hips"],
  ["fever", "sweats", "cough", "hemoptysis"],
  ["cancer", "lumpectomy", "chemotherapy"],
  ["radiation", "therapy", "tamoxifen"],
  ["family", "ovarian", "cancer"],
  ["pallor", "lymphadenopathy", "supraclavicular", "tenderness", "lumbar", "spine"],
  ["vital", "fever"],
  ["neurological", "motor", "sensory"],
  ["skin", "petechiae", "extremities"],
  ["anemia", "hemoglobin", "alkaline", "phosphatase"],
  ["calcium", "renal", "function"],
  ["liver", "enzymes", "transaminases"],
  ["tumor", "markers", "ca15-3", "cea"],
  ["bone", "marrow", "biopsy", "pancytopenia"],
  ["radiographs", "vertebral", "lytic", "lesions"],
  ["bone", "scan", "metastases"],
  ["ct", "chest", "abdomen", "pelvis", "lesions"],
  ["pulmonary", "nodules", "lymphadenopathy"],
  ["biopsy", "adenocarcinoma", "breast", "primary"],
  ["immunohistochemistry", "estrogen", "progesterone", "her2"],
  ["bisphosphonates", "aromatase", "inhibitors"],
  ["analgesics", "hypercalcemia", "hydration"],
  ["pain", "management", "physiotherapy"],
  ["counseling", "prognosis", "quality", "life"],
  ["imaging", "therapy", "response"],
  ["hemoglobin", "bisphosphonate", "therapy"],
  ["tumor", "markers", "trend"],
  ["nausea", "fatigue", "aromatase"],
  ["endocrinology", "bone", "density", "calcium"],
  ["psychological", "anxiety", "mood"],
  ["genetic", "counseling", "therapy"],
  ["vitamin", "d", "supplementation"],
  ["liver", "function", "hematologic"],
  ["brain", "metastases", "mri", "headaches"],
  ["tumor", "board", "management"],
  ["rehabilitation", "mobility", "complications"],
  ["diet", "nutritional", "deficits"],
  ["fractures", "orthopedics"],
  ["patient", "goals", "symptom", "control"],
  ["anemia", "erythropoiesis"],
  ["side-effects", "arthralgia", "hormonal"],
  ["bone", "biopsies", "progression"],
  ["imaging", "surveillance"],
  ["clinical", "trial", "breast", "cancer"],
  ["adherence", "nursing", "education"],
  ["palliative", "care", "support"],
  ["thrombocytopenia", "dose", "adjustments"],
  ["cardiotoxicity", "echocardiogram", "chemotherapy"],
  ["psychiatric", "intervention", "mood"],
  ["care", "planning", "family"],
  ["support", "groups", "oncology"],
  ["analgesic", "opioids", "nsaids"],
  ["bone-modifying", "skeletal", "quality"],
  ["biomarkers", "monitoring"],
  ["hormonal", "therapy", "calcium", "vitamin"],
  ["fatigue", "activity", "conservation"],
  ["disease", "progression", "resistance"],
  ["communication", "oncology", "rehabilitation", "social"],
  ["financial", "support", "treatment"],
  ["symptoms", "side-effects", "diary"],
  ["goals", "care", "status"],
  ["guidelines", "management"],
  ["balance", "control", "toxicity"],
  ["activities", "social", "interaction"],
  ["survivorship", "monitoring"],
  ["education", "trainees", "cancer"],
  ["care", "transitions", "community"],
  ["decision-making", "family", "prognosis"],
  ["documentation", "medication", "allergy"],
  ["research", "quality", "improvement"],
  ["diagnosis", "breast", "cancer", "metastatic", "skeletal", "hepatic"],
  ["prognosis", "communication", "care"]
]



}

from itertools import product
import time

# === HYPERPARAMETER GRID SEARCH ===
def grid_search_best_params(chunks, entities_per_chunk, umap_param_grid, hdbscan_param_grid):
    best_score = -np.inf
    best_umap = None
    best_hdbscan = None
    best_metrics = None

    combinations = list(product(umap_param_grid["n_neighbors"],
                                umap_param_grid["n_components"],
                                umap_param_grid["min_dist"],
                                hdbscan_param_grid["min_cluster_size"],
                                hdbscan_param_grid["min_samples"]))

    logger.info(f"Starting grid search over {len(combinations)} combinations...")

    for (n_neighbors, n_components, min_dist, min_cluster_size, min_samples) in combinations:
        umap_params = {
            "n_neighbors": n_neighbors,
            "n_components": n_components,
            "min_dist": min_dist,
            "metric": "cosine"
        }
        hdbscan_params = {
            "min_cluster_size": min_cluster_size,
            "min_samples": min_samples,
            "metric": "euclidean"
        }

        try:
            start_time = time.time()
            model = AllergyTopicSearcher(chunks, entities_per_chunk, umap_params, hdbscan_params)
            coherence = compute_bertopic_coherence(model.topic_model, model.topic_metadata)
            diversity = compute_topic_diversity(model.topic_model, model.topic_metadata)
            silhouette = compute_silhouette_score_custom(model.topic_metadata) or 0.0

            # Composite scoring (tweak weights if desired)
            score = coherence + 0.5 * diversity + 0.5 * silhouette
            elapsed = time.time() - start_time

            logger.info(
                f" Tried UMAP: {umap_params}, HDBSCAN: {hdbscan_params} "
                f"=> Score: {score:.4f} (Coherence: {coherence:.4f}, Diversity: {diversity:.4f}, Silhouette: {silhouette:.4f}) - Time: {elapsed:.2f}s"
            )

            if score > best_score:
                best_score = score
                best_umap = umap_params
                best_hdbscan = hdbscan_params
                best_metrics = (coherence, diversity, silhouette)

        except Exception as e:
            logger.warning(f"Skipping combination due to error: {e}")

    return best_umap, best_hdbscan, best_metrics
# Define parameter grids
umap_grid = {
    "n_neighbors": [3, 5, 8],
    "n_components": [3, 5],
    "min_dist": [0.0, 0.1, 0.3]
}

hdbscan_grid = {
    "min_cluster_size": [2, 5],
    "min_samples": [1, 3]
}

# Run grid search
best_umap, best_hdbscan, best_metrics = grid_search_best_params(
    allergy_dataset["chunks"],
    allergy_dataset["entities"],
    umap_param_grid=umap_grid,
    hdbscan_param_grid=hdbscan_grid
)

print("\n=== Best Hyperparameters Found ===")
print(f"Best UMAP: {best_umap}")
print(f"Best HDBSCAN: {best_hdbscan}")
print(f"Metrics => Coherence: {best_metrics[0]:.4f}, Diversity: {best_metrics[1]:.4f}, Silhouette: {best_metrics[2]:.4f}")


searcher = AllergyTopicSearcher(
    chunks=allergy_dataset["chunks"],
    entities_per_chunk=allergy_dataset["entities"],
    umap_params=best_umap,
    hdbscan_params=best_hdbscan,
    model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)
print("Model ready for querying.")
# print("\n=== Generated Topics and Entities ===")
# for meta in searcher.topic_metadata:
#     topic_id = meta["topic_id"]
#     entities = ", ".join(meta["entities"])
#     print(f"Topic ID: {topic_id} — Entities: {entities}")

# === METRICS ===
coherence = compute_bertopic_coherence(searcher.topic_model, searcher.topic_metadata, topk=15)
diversity = compute_topic_diversity(searcher.topic_model, searcher.topic_metadata, topk=10)
sil_score = compute_silhouette_score_custom(searcher.topic_metadata)

print("\n=== Topic Quality Metrics ===")
print(f"Coherence Score (c_v): {coherence:.4f}")
print(f"Topic Diversity: {diversity:.4f}")
if sil_score is not None:
    print(f" Silhouette Score: {sil_score:.4f}")
else:
    print("Silhouette Score: Not applicable.")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



=== Best Hyperparameters Found ===
Best UMAP: {'n_neighbors': 5, 'n_components': 5, 'min_dist': 0.1, 'metric': 'cosine'}
Best HDBSCAN: {'min_cluster_size': 2, 'min_samples': 1, 'metric': 'euclidean'}
Metrics => Coherence: 0.7154, Diversity: 0.4153, Silhouette: 0.7765
Model ready for querying.

=== Topic Quality Metrics ===
Coherence Score (c_v): 0.7154
Topic Diversity: 0.4153
 Silhouette Score: 0.7765


In [8]:
# === IMPORTS & SETUP ===
import os
import random
import numpy as np
import torch
import nltk
import logging
import re

from collections import defaultdict, Counter
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP
import scann

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.metrics import silhouette_score, precision_recall_fscore_support
from sklearn.metrics.pairwise import cosine_similarity

# === SEED FIXING ===
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.manual_seed_all(SEED)
torch.use_deterministic_algorithms(True)
nltk.download("punkt")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# === CLEANING & CONTEXT EXTRACTION (IMPROVED) ===
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return re.sub(r"\s+", " ", text).strip()


def extract_entity_contexts(chunks, entities_per_chunk, use_multi_sentence=True):
    entity_context_pairs = []
    for idx, ents in enumerate(entities_per_chunk):
        chunk = clean_text(chunks[idx])
        sentences = sent_tokenize(chunk)
        for ent in ents:
            ent_lower = ent.lower()
            matched = False
            for i, sent in enumerate(sentences):
                if ent_lower in sent:
                    context = " ".join(sentences[max(0, i - 1): i + 2]) if use_multi_sentence else sent.strip()
                    enriched = f"The concept '{ent_lower}' appears in the following context: {context}"
                    entity_context_pairs.append((ent_lower, enriched.strip()))
                    matched = True
                    break
            if not matched:
                fallback = f"The concept '{ent_lower}' appears in the following context: {chunk}"
                entity_context_pairs.append((ent_lower, fallback.strip()))
    return entity_context_pairs


# === TOPIC SEARCHER CLASS (WITH DEDUPLICATION, NOISE FILTERING) ===
class AllergyTopicSearcher:
    def __init__(self, chunks, entities_per_chunk, umap_params, hdbscan_params,
                 model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"):
        self.chunks = chunks
        self.entities_per_chunk = entities_per_chunk
        self.embedding_model = SentenceTransformer(model_name)

        self.umap_params = umap_params
        self.hdbscan_params = hdbscan_params

        self.topic_model = None
        self.topic_metadata = []
        self.topic_embeddings = None
        self.searcher = None

        self._prepare()

    def _prepare(self):
        entity_context_pairs = extract_entity_contexts(self.chunks, self.entities_per_chunk)
        contextual_texts = [ctx for _, ctx in entity_context_pairs]
        contextual_embeddings = self.embedding_model.encode(contextual_texts, normalize_embeddings=False)

        umap_model = UMAP(**self.umap_params, random_state=SEED)
        hdbscan_model = HDBSCAN(**self.hdbscan_params, prediction_data=True)

        self.topic_model = BERTopic(
            embedding_model=self.embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            representation_model=KeyBERTInspired(),
            calculate_probabilities=True,
            verbose=False,
        )

        topics, _ = self.topic_model.fit_transform(contextual_texts, embeddings=contextual_embeddings)

        topic_to_contexts = defaultdict(list)
        topic_to_entities = defaultdict(set)
        topic_to_embeddings = defaultdict(list)

        for i, topic in enumerate(topics):
            if topic == -1:
                continue  # Skip noisy topics
            ent, ctx = entity_context_pairs[i]
            topic_to_contexts[topic].append(ctx)
            topic_to_entities[topic].add(ent)
            topic_to_embeddings[topic].append(contextual_embeddings[i])

        topic_embeddings = []
        topic_metadata = []

        for topic_id in topic_to_contexts:
            emb = topic_to_embeddings[topic_id]
            centroid = np.mean(emb, axis=0)
            centroid /= np.linalg.norm(centroid) + 1e-10
            topic_embeddings.append(centroid)
            topic_metadata.append({
                "topic_id": topic_id,
                "entities": list(topic_to_entities[topic_id]),
                "sentences": topic_to_contexts[topic_id],
                "sentence_embeddings": np.array(emb)
            })

        # === OPTIONAL: Merge semantically similar topics (cosine sim > 0.95)
        deduped_metadata = []
        used = set()

        for i, emb_i in enumerate(topic_embeddings):
            if i in used:
                continue
            group = [i]
            sim_scores = cosine_similarity([emb_i], topic_embeddings)[0]
            for j in range(i + 1, len(sim_scores)):
                if sim_scores[j] > 0.95:
                    group.append(j)
                    used.add(j)

            merged = {
                "topic_id": i,
                "sentences": [],
                "entities": [],
                "sentence_embeddings": []
            }
            for g in group:
                merged["sentences"] += topic_metadata[g]["sentences"]
                merged["entities"] += topic_metadata[g]["entities"]
                merged["sentence_embeddings"] += list(topic_metadata[g]["sentence_embeddings"])

            merged["sentence_embeddings"] = np.array(merged["sentence_embeddings"])
            merged["entities"] = list(set(merged["entities"]))
            deduped_metadata.append(merged)

        self.topic_metadata = deduped_metadata
        self.topic_embeddings = np.array([
            np.mean(m["sentence_embeddings"], axis=0) /
            (np.linalg.norm(np.mean(m["sentence_embeddings"], axis=0)) + 1e-10)
            for m in deduped_metadata
        ])

        num_points = len(self.topic_embeddings)
        num_leaves = min(10, num_points)
        leaves_to_search = min(5, num_leaves)

        if num_points < 3:
            raise ValueError("Not enough topics to build a search index.")

        self.searcher = (
            scann.scann_ops_pybind.builder(self.topic_embeddings, 3, "dot_product")
            .tree(num_leaves=num_leaves, num_leaves_to_search=leaves_to_search, training_sample_size=num_points)
            .score_brute_force()
            .reorder(min(5, num_points))
            .build()
        )


    import re

    def search(self, query, top_k_topics=3, top_k_sents=3):
        query_emb = self.embedding_model.encode([query], normalize_embeddings=True)[0]
        neighbors, scores = self.searcher.search(query_emb, final_num_neighbors=top_k_topics)

        results = []
        prefix_pattern = r"^the concept '.*?' appears in (the following )?context:\s*"

        for i, idx in enumerate(neighbors):
            meta = self.topic_metadata[idx]
            topic_score = float(scores[i])

            # Deduplicate sentences
            seen = set()
            cleaned_sentences = []
            cleaned_embeddings = []

            for sent, emb in zip(meta["sentences"], meta["sentence_embeddings"]):
            # Apply regex to remove beginning prefix
                cleaned = re.sub(prefix_pattern, "", sent, flags=re.IGNORECASE).strip()

        # No duplicates
                if cleaned not in seen:
                    seen.add(cleaned)
                    cleaned_sentences.append(cleaned)
                    cleaned_embeddings.append(emb)

            if not cleaned_sentences:
                continue

            emb_array = np.array(cleaned_embeddings)
            sims = np.dot(emb_array / np.linalg.norm(emb_array, axis=1, keepdims=True), query_emb)
            top_ids = sims.argsort()[::-1][:top_k_sents]

            top_sents = [(cleaned_sentences[j], float(sims[j])) for j in top_ids]
            results.append({
            "topic_id": meta["topic_id"],
            "topic_score": topic_score,
            "entities": meta["entities"],
            "sentences": top_sents,
            })

        return results




# === METRICS ===
def compute_bertopic_coherence(topic_model, topic_metadata, topk=15):
    topics = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    word_lists = [[word for word, _ in topic] for topic in topics]

    texts = []
    for meta in topic_metadata:
        for s in meta["sentences"]:
            tokens = clean_text(s).split()
            texts.append(tokens)

    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(t) for t in texts]

    cm = CoherenceModel(
        topics=word_lists,
        texts=texts,
        dictionary=dictionary,
        coherence="c_v"
    )
    return cm.get_coherence()


def compute_topic_diversity(topic_model, topic_metadata, topk=10):
    topic_words = [topic_model.get_topic(meta["topic_id"])[:topk] for meta in topic_metadata]
    unique_words = set(word for topic in topic_words for word, _ in topic)
    return len(unique_words) / (len(topic_words) * topk)


def compute_silhouette_score_custom(topic_metadata):
    all_embeddings = []
    all_labels = []

    for meta in topic_metadata:
        emb = meta["sentence_embeddings"]
        if len(emb) < 2:  # skip small clusters
            continue
        all_embeddings.extend(emb)
        all_labels.extend([meta["topic_id"]] * len(emb))

    if len(all_embeddings) < 3:
        return None

    all_embeddings = np.vstack(all_embeddings)
    return silhouette_score(all_embeddings, all_labels, metric="cosine")


# === DATASET & INITIALIZATION ===
allergy_dataset = {
  "chunks":  [
  "A 75-year-old male with ischemic cardiomyopathy (LVEF 28%) and persistent atrial fibrillation on warfarin presented with worsening dyspnea and orthopnea over 2 weeks.",
  "He has a history of COPD, stage 4 chronic kidney disease, type 2 diabetes mellitus (HbA1c 9.1%), and prior CABG in 2008.",
  "On admission, his vitals were BP 155/95, HR 112 irregular, RR 28, and SpO₂ 86% on room air, improving with oxygen via nasal cannula.",
  "Physical exam revealed elevated JVP, bibasilar crackles, 3+ pitting edema, and an S3 gallop.",
  "Labs showed Cr 2.8, eGFR 25, BNP 2,400 pg/mL, and INR 2.3. ABG showed pH 7.36, PaCO₂ 50, PaO₂ 70. CXR indicated cardiomegaly and pleural effusions.",
  "ECG confirmed atrial fibrillation with QTc 480 ms. Pacemaker function was intact.",
  "He was treated with IV furosemide, oxygen therapy, and continued on carvedilol and lisinopril. Metformin was held due to renal impairment.",
  "Diabetes was managed with basal-bolus insulin and sliding scale. A low-sodium renal diet was started.",
  "Education included fluid restriction, daily weight tracking, and signs of heart failure. Warfarin was continued with INR monitoring.",
  "He showed improvement with a 3 kg net negative fluid balance and SpO₂ rising to 94%.",
  "An echocardiogram showed LVEF 28%, moderate mitral regurgitation, and left atrial enlargement.",
  "Discharge medications included furosemide 80 mg, carvedilol 12.5 mg BID, lisinopril 5 mg, spironolactone 25 mg, basal insulin, and warfarin.",
  "Follow-up appointments were scheduled with cardiology, nephrology, and the diabetes clinic. Anticoagulation clinic and home health visits arranged.",
  "Patient is a widower living alone with limited mobility and uses a walker. Transportation issues were noted.",
  "Social work coordinated support services including meal delivery, medication management, and transport assistance."
]
,
"entities":[
  ["male", "cardiomyopathy", "lvef", "fibrillation", "warfarin", "dyspnea", "orthopnea"],
  ["copd", "kidney", "disease", "diabetes", "hba1c", "cabg"],
  ["bp", "hr", "irregularity", "rr", "spo2", "oxygen", "nasal", "cannula"],
  ["jvp", "crackles", "edema", "gallop"],
  ["cr", "egfr", "bnp", "inr", "abg", "ph", "paco2", "pao2", "cxr", "cardiomegaly", "effusions"],
  ["ecg", "fibrillation", "qtc", "pacemaker"],
  ["furosemide", "oxygen", "therapy", "carvedilol", "lisinopril", "metformin", "renal", "impairment"],
  ["diabetes", "insulin", "scale", "diet"],
  ["fluid", "weight", "tracking", "signs", "failure", "warfarin", "inr", "monitoring"],
  ["improvement", "fluid", "balance", "spo2"],
  ["echocardiogram", "lvef", "regurgitation", "atrium", "enlargement"],
  ["furosemide", "carvedilol", "lisinopril", "spironolactone", "insulin", "warfarin"],
  ["follow-up", "cardiology", "nephrology", "clinic", "anticoagulation", "health", "visits"],
  ["widower", "mobility", "walker"],
  ["social", "support", "meal", "delivery", "medication", "management", "transport"]
]



}

from itertools import product
import time

# === HYPERPARAMETER GRID SEARCH ===
def grid_search_best_params(chunks, entities_per_chunk, umap_param_grid, hdbscan_param_grid):
    best_score = -np.inf
    best_umap = None
    best_hdbscan = None
    best_metrics = None

    combinations = list(product(umap_param_grid["n_neighbors"],
                                umap_param_grid["n_components"],
                                umap_param_grid["min_dist"],
                                hdbscan_param_grid["min_cluster_size"],
                                hdbscan_param_grid["min_samples"]))

    logger.info(f"Starting grid search over {len(combinations)} combinations...")

    for (n_neighbors, n_components, min_dist, min_cluster_size, min_samples) in combinations:
        umap_params = {
            "n_neighbors": n_neighbors,
            "n_components": n_components,
            "min_dist": min_dist,
            "metric": "cosine"
        }
        hdbscan_params = {
            "min_cluster_size": min_cluster_size,
            "min_samples": min_samples,
            "metric": "euclidean"
        }

        try:
            start_time = time.time()
            model = AllergyTopicSearcher(chunks, entities_per_chunk, umap_params, hdbscan_params)
            coherence = compute_bertopic_coherence(model.topic_model, model.topic_metadata)
            diversity = compute_topic_diversity(model.topic_model, model.topic_metadata)
            silhouette = compute_silhouette_score_custom(model.topic_metadata) or 0.0

            # Composite scoring (tweak weights if desired)
            score = coherence + 0.5 * diversity + 0.5 * silhouette
            elapsed = time.time() - start_time

            logger.info(
                f" Tried UMAP: {umap_params}, HDBSCAN: {hdbscan_params} "
                f"=> Score: {score:.4f} (Coherence: {coherence:.4f}, Diversity: {diversity:.4f}, Silhouette: {silhouette:.4f}) - Time: {elapsed:.2f}s"
            )

            if score > best_score:
                best_score = score
                best_umap = umap_params
                best_hdbscan = hdbscan_params
                best_metrics = (coherence, diversity, silhouette)

        except Exception as e:
            logger.warning(f"Skipping combination due to error: {e}")

    return best_umap, best_hdbscan, best_metrics
# Define parameter grids
umap_grid = {
    "n_neighbors": [3, 5, 8],
    "n_components": [3, 5],
    "min_dist": [0.0, 0.1, 0.3]
}

hdbscan_grid = {
    "min_cluster_size": [2, 5],
    "min_samples": [1, 3]
}

# Run grid search
best_umap, best_hdbscan, best_metrics = grid_search_best_params(
    allergy_dataset["chunks"],
    allergy_dataset["entities"],
    umap_param_grid=umap_grid,
    hdbscan_param_grid=hdbscan_grid
)

print("\n=== Best Hyperparameters Found ===")
print(f"Best UMAP: {best_umap}")
print(f"Best HDBSCAN: {best_hdbscan}")
print(f"Metrics => Coherence: {best_metrics[0]:.4f}, Diversity: {best_metrics[1]:.4f}, Silhouette: {best_metrics[2]:.4f}")


searcher = AllergyTopicSearcher(
    chunks=allergy_dataset["chunks"],
    entities_per_chunk=allergy_dataset["entities"],
    umap_params=best_umap,
    hdbscan_params=best_hdbscan,
    model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
)
print("Model ready for querying.")
# print("\n=== Generated Topics and Entities ===")
# for meta in searcher.topic_metadata:
#     topic_id = meta["topic_id"]
#     entities = ", ".join(meta["entities"])
#     print(f"Topic ID: {topic_id} — Entities: {entities}")

# === METRICS ===
coherence = compute_bertopic_coherence(searcher.topic_model, searcher.topic_metadata, topk=15)
diversity = compute_topic_diversity(searcher.topic_model, searcher.topic_metadata, topk=10)
sil_score = compute_silhouette_score_custom(searcher.topic_metadata)

print("\n=== Topic Quality Metrics ===")
print(f"Coherence Score (c_v): {coherence:.4f}")
print(f"Topic Diversity: {diversity:.4f}")
if sil_score is not None:
    print(f" Silhouette Score: {sil_score:.4f}")
else:
    print("Silhouette Score: Not applicable.")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



=== Best Hyperparameters Found ===
Best UMAP: {'n_neighbors': 3, 'n_components': 5, 'min_dist': 0.3, 'metric': 'cosine'}
Best HDBSCAN: {'min_cluster_size': 5, 'min_samples': 1, 'metric': 'euclidean'}
Metrics => Coherence: 0.7843, Diversity: 0.7818, Silhouette: 0.6811
Model ready for querying.

=== Topic Quality Metrics ===
Coherence Score (c_v): 0.7843
Topic Diversity: 0.7818
 Silhouette Score: 0.6811
